In [1]:
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 237kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('/content/drive/My Drive/Computer_vision_project/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [2]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [6]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2

Num GPUs Available:  0


In [0]:
# def Model(num_classes):
#   layers = [
    
#     #  tf.keras.layers.BatchNormalization(),
#       # tf.keras.layers.Dense(256, activation = tf.nn.relu),
#       # tf.keras.layers.LSTM(64),

#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(128, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(64, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(32, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
      
#       tf.keras.layers.Dense(num_classes, activation = 'softmax')
#   ]
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
def build_model(row,col,num_classes):

  tf.keras.initializers.GlorotNormal(234)
  input_layer = tf.keras.Input(shape=(row, col))
  
  # layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(input_layer)
 
  
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(input_layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layer = tf.keras.layers.LSTM(16)(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
 #
 
  # layerconv2 = tf.keras.layers.Conv1D(filters=256,kernel_size= 3,strides=5) (input_layer)
  # layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  # layerconv3 = tf.keras.layers.Conv1D(filters=128,kernel_size= 5,strides=2) (input_layer)
  # layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)


  layerconv1 = tf.keras.layers.Conv1D(filters=30,kernel_size=5,  strides=2) (input_layer)
  layerconv1 = tf.keras.layers.Activation('relu')(layerconv1)
  layerconv2 = tf.keras.layers.Conv1D(filters=40,kernel_size= 3,strides=5) (input_layer)
  layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  layerconv3 = tf.keras.layers.Conv1D(filters=50,kernel_size= 2,strides=7) (input_layer)
  layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)
  layerconv4 = tf.keras.layers.Conv1D(filters=60,kernel_size= 1,strides=5) (input_layer)
  layerconv4 = tf.keras.layers.Activation('relu')(layerconv4)
  max1= tf.reduce_max(layerconv1, 1)
  max2= tf.reduce_max(layerconv2, 1)
  max3= tf.reduce_max(layerconv3, 1)
  max4= tf.reduce_max(layerconv4, 1)
  # concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4],1)
 
 
  # max3= tf.reduce_max(layerconv2, 1)
  # max4= tf.reduce_max(layerconv3, 1)
  concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4,layer],1)
  layer = tf.keras.layers.Dense(64, activation = tf.nn.relu)(concat_layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layer=    tf.keras.layers.Dense(32, activation = tf.nn.relu)(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (layer) 

    
 
  # layers = [
     
  #      tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
          
        
  #       tf.keras.layers.LSTM(28,return_sequences=True,  input_shape=(row, col)),
  #       tf.keras.layers.Dropout(0.2),
  #       tf.keras.layers.LSTM(64,return_sequences=True), 
  #       tf.keras.layers.Dropout(0.2),
  #       tf.keras.layers.Conv1D(filters=30,kernel_size=5,strides=1),
        
  #       tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1),
  #       tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1),
  #       tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1),
  #       tf.keras.layers.Dropout(0.2),

        
                       

       
        
       
  #       tf.keras.layers.Dense(num_classes, activation = 'softmax')
  #   ]
    
  model = tf.keras.Model(inputs=input_layer, outputs=layer)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, y_train.shape)
  print(X_test.shape,y_test.shape)
  size, rows = X_train.shape
  X_train=X_train.reshape(X_train.shape[0],rows,1)
  # y_train=y_train.reshape(y_train.shape[0],num_classes)
  print(X_train.shape)
  # _,img_rows, img_cols,_ = X.shape

  model = build_model(rows,1,num_classes)
  # es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=50,restore_best_weights=True)
  optimizer = tf.keras.optimizers.Adam(lr=0.001)
    # callbacks = [
    #     tf.keras.callbacks.EarlyStopping(patience=stopping_patience, restore_best_weights=True),
        
    # ]

  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=128, epochs=500)
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [32]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
(1711, 22, 1)
Epoch 1/500
14/14 [==============================] - 0s 16ms/step - loss: 2.3060 - sparse_categorical_accuracy: 0.2385
Epoch 2/500
14/14 [==============================] - 0s 13ms/step - loss: 1.8701 - sparse_categorical_accuracy: 0.3495
Epoch 3/500
14/14 [==============================] - 0s 13ms/step - loss: 1.5288 - sparse_categorical_accuracy: 0.4816
Epoch 4/500
14/14 [==============================] - 0s 13ms/step - loss: 1.2638 - sparse_categorical_accuracy: 0.5669
Epoch 5/500
14/14 [==============================] - 0s 13ms/step - loss: 1.0402 - sparse_categorical_accuracy: 0.6417
Epoch 6/500
14/14 [==============================] - 0s 13ms/step - loss: 0.8832 - sparse_categorical_accuracy: 0.6885
Epoch 7/500
14/14 [==============================] - 0s 13ms/step - loss: 0.7932 - sparse_categorical_accuracy: 0.7247
Epoch 8/500
14/14 [==============================] - 0s 13ms/step - loss: 0.6

In [33]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
(1711, 22, 1)
Epoch 1/500
14/14 [==============================] - 0s 13ms/step - loss: 2.1640 - sparse_categorical_accuracy: 0.2227
Epoch 2/500
14/14 [==============================] - 0s 13ms/step - loss: 1.8254 - sparse_categorical_accuracy: 0.3612
Epoch 3/500
14/14 [==============================] - 0s 13ms/step - loss: 1.5790 - sparse_categorical_accuracy: 0.4693
Epoch 4/500
14/14 [==============================] - 0s 13ms/step - loss: 1.3615 - sparse_categorical_accuracy: 0.5412
Epoch 5/500
14/14 [==============================] - 0s 13ms/step - loss: 1.1473 - sparse_categorical_accuracy: 0.6148
Epoch 6/500
14/14 [==============================] - 0s 13ms/step - loss: 0.9750 - sparse_categorical_accuracy: 0.6575
Epoch 7/500
14/14 [==============================] - 0s 14ms/step - loss: 0.8366 - sparse_categorical_accuracy: 0.7200
Epoch 8/500
14/14 [==============================] - 0s 13ms/step - loss: 0.7

In [34]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(894, 22) (894,)
(441, 22) (441,)
(894, 22, 1)
Epoch 1/500
7/7 [==============================] - 0s 18ms/step - loss: 2.3653 - sparse_categorical_accuracy: 0.1588
Epoch 2/500
7/7 [==============================] - 0s 12ms/step - loss: 2.0388 - sparse_categorical_accuracy: 0.3031
Epoch 3/500
7/7 [==============================] - 0s 12ms/step - loss: 1.8310 - sparse_categorical_accuracy: 0.4027
Epoch 4/500
7/7 [==============================] - 0s 12ms/step - loss: 1.6731 - sparse_categorical_accuracy: 0.4430
Epoch 5/500
7/7 [==============================] - 0s 12ms/step - loss: 1.5916 - sparse_categorical_accuracy: 0.4676
Epoch 6/500
7/7 [==============================] - 0s 12ms/step - loss: 1.4759 - sparse_categorical_accuracy: 0.5190
Epoch 7/500
7/7 [==============================] - 0s 12ms/step - loss: 1.3777 - sparse_categorical_accuracy: 0.5246
Epoch 8/500
7/7 [==============================] - 0s 12ms/step - loss: 1.2810 - sparse_catego

In [35]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
(1711, 22, 1)
Epoch 1/500
14/14 [==============================] - 0s 13ms/step - loss: 2.2299 - sparse_categorical_accuracy: 0.2279
Epoch 2/500
14/14 [==============================] - 0s 12ms/step - loss: 1.7360 - sparse_categorical_accuracy: 0.3974
Epoch 3/500
14/14 [==============================] - 0s 12ms/step - loss: 1.4827 - sparse_categorical_accuracy: 0.5044
Epoch 4/500
14/14 [==============================] - 0s 13ms/step - loss: 1.2688 - sparse_categorical_accuracy: 0.5564
Epoch 5/500
14/14 [==============================] - 0s 13ms/step - loss: 1.0637 - sparse_categorical_accuracy: 0.6148
Epoch 6/500
14/14 [==============================] - 0s 12ms/step - loss: 0.8664 - sparse_categorical_accuracy: 0.7002
Epoch 7/500
14/14 [==============================] - 0s 13ms/step - loss: 0.7573 - sparse_categorical_accuracy: 0.7534
Epoch 8/500
14/14 [==============================] - 0s 12ms/step - loss: 0.6

In [36]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
(6582, 7, 1)
Epoch 1/500
52/52 [==============================] - 1s 10ms/step - loss: 1.9367 - sparse_categorical_accuracy: 0.3526
Epoch 2/500
52/52 [==============================] - 0s 9ms/step - loss: 1.1974 - sparse_categorical_accuracy: 0.5403
Epoch 3/500
52/52 [==============================] - 0s 9ms/step - loss: 0.9512 - sparse_categorical_accuracy: 0.6203
Epoch 4/500
52/52 [==============================] - 0s 9ms/step - loss: 0.8546 - sparse_categorical_accuracy: 0.6551
Epoch 5/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7814 - sparse_categorical_accuracy: 0.6832
Epoch 6/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7485 - sparse_categorical_accuracy: 0.6905
Epoch 7/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7099 - sparse_categorical_accuracy: 0.7072
Epoch 8/500
52/52 [==============================] - 0s 9ms/step - loss: 0.6732 - spa

In [37]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
(6582, 7, 1)
Epoch 1/500
52/52 [==============================] - 0s 9ms/step - loss: 1.9836 - sparse_categorical_accuracy: 0.3122
Epoch 2/500
52/52 [==============================] - 0s 9ms/step - loss: 1.3685 - sparse_categorical_accuracy: 0.4988
Epoch 3/500
52/52 [==============================] - 0s 9ms/step - loss: 1.0430 - sparse_categorical_accuracy: 0.5881
Epoch 4/500
52/52 [==============================] - 0s 9ms/step - loss: 0.8843 - sparse_categorical_accuracy: 0.6396
Epoch 5/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7976 - sparse_categorical_accuracy: 0.6831
Epoch 6/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7522 - sparse_categorical_accuracy: 0.6867
Epoch 7/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7112 - sparse_categorical_accuracy: 0.7071
Epoch 8/500
52/52 [==============================] - 0s 9ms/step - loss: 0.6723 - spar

In [38]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(3375, 7) (3375,)
(1663, 7) (1663,)
(3375, 7, 1)
Epoch 1/500
27/27 [==============================] - 0s 11ms/step - loss: 2.1918 - sparse_categorical_accuracy: 0.2273
Epoch 2/500
27/27 [==============================] - 0s 9ms/step - loss: 1.7141 - sparse_categorical_accuracy: 0.4199
Epoch 3/500
27/27 [==============================] - 0s 9ms/step - loss: 1.3689 - sparse_categorical_accuracy: 0.5105
Epoch 4/500
27/27 [==============================] - 0s 9ms/step - loss: 1.1408 - sparse_categorical_accuracy: 0.5653
Epoch 5/500
27/27 [==============================] - 0s 9ms/step - loss: 1.0136 - sparse_categorical_accuracy: 0.6012
Epoch 6/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9427 - sparse_categorical_accuracy: 0.6237
Epoch 7/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8721 - sparse_categorical_accuracy: 0.6521
Epoch 8/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8311 - spa

In [39]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
(6582, 7, 1)
Epoch 1/500
52/52 [==============================] - 0s 9ms/step - loss: 1.8636 - sparse_categorical_accuracy: 0.3472
Epoch 2/500
52/52 [==============================] - 0s 9ms/step - loss: 1.2295 - sparse_categorical_accuracy: 0.5334
Epoch 3/500
52/52 [==============================] - 0s 9ms/step - loss: 0.9830 - sparse_categorical_accuracy: 0.5995
Epoch 4/500
52/52 [==============================] - 0s 9ms/step - loss: 0.8595 - sparse_categorical_accuracy: 0.6474
Epoch 5/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7753 - sparse_categorical_accuracy: 0.6785
Epoch 6/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7356 - sparse_categorical_accuracy: 0.7012
Epoch 7/500
52/52 [==============================] - 0s 9ms/step - loss: 0.7099 - sparse_categorical_accuracy: 0.7110
Epoch 8/500
52/52 [==============================] - 0s 9ms/step - loss: 0.6764 - spar

In [40]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
(2593, 5, 1)
Epoch 1/500
21/21 [==============================] - 0s 10ms/step - loss: 2.1705 - sparse_categorical_accuracy: 0.3428
Epoch 2/500
21/21 [==============================] - 0s 8ms/step - loss: 1.7961 - sparse_categorical_accuracy: 0.4431
Epoch 3/500
21/21 [==============================] - 0s 8ms/step - loss: 1.6642 - sparse_categorical_accuracy: 0.4724
Epoch 4/500
21/21 [==============================] - 0s 8ms/step - loss: 1.5659 - sparse_categorical_accuracy: 0.4836
Epoch 5/500
21/21 [==============================] - 0s 8ms/step - loss: 1.5015 - sparse_categorical_accuracy: 0.4979
Epoch 6/500
21/21 [==============================] - 0s 9ms/step - loss: 1.4545 - sparse_categorical_accuracy: 0.5033
Epoch 7/500
21/21 [==============================] - 0s 8ms/step - loss: 1.3817 - sparse_categorical_accuracy: 0.5276
Epoch 8/500
21/21 [==============================] - 0s 8ms/step - loss: 1.3589 - spar

In [41]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
(2593, 5, 1)
Epoch 1/500
21/21 [==============================] - 0s 8ms/step - loss: 2.2278 - sparse_categorical_accuracy: 0.3051
Epoch 2/500
21/21 [==============================] - 0s 9ms/step - loss: 1.7963 - sparse_categorical_accuracy: 0.4354
Epoch 3/500
21/21 [==============================] - 0s 9ms/step - loss: 1.6236 - sparse_categorical_accuracy: 0.4663
Epoch 4/500
21/21 [==============================] - 0s 8ms/step - loss: 1.5325 - sparse_categorical_accuracy: 0.4705
Epoch 5/500
21/21 [==============================] - 0s 9ms/step - loss: 1.4578 - sparse_categorical_accuracy: 0.4952
Epoch 6/500
21/21 [==============================] - 0s 9ms/step - loss: 1.3874 - sparse_categorical_accuracy: 0.5183
Epoch 7/500
21/21 [==============================] - 0s 9ms/step - loss: 1.3438 - sparse_categorical_accuracy: 0.5137
Epoch 8/500
21/21 [==============================] - 0s 9ms/step - loss: 1.3153 - spars

In [42]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(2599, 5) (2599,)
(1281, 5) (1281,)
(2599, 5, 1)
Epoch 1/500
21/21 [==============================] - 0s 10ms/step - loss: 2.2676 - sparse_categorical_accuracy: 0.2189
Epoch 2/500
21/21 [==============================] - 0s 7ms/step - loss: 1.7864 - sparse_categorical_accuracy: 0.4459
Epoch 3/500
21/21 [==============================] - 0s 8ms/step - loss: 1.6209 - sparse_categorical_accuracy: 0.4629
Epoch 4/500
21/21 [==============================] - 0s 8ms/step - loss: 1.4812 - sparse_categorical_accuracy: 0.4871
Epoch 5/500
21/21 [==============================] - 0s 8ms/step - loss: 1.4378 - sparse_categorical_accuracy: 0.4990
Epoch 6/500
21/21 [==============================] - 0s 9ms/step - loss: 1.3827 - sparse_categorical_accuracy: 0.5125
Epoch 7/500
21/21 [==============================] - 0s 8ms/step - loss: 1.3372 - sparse_categorical_accuracy: 0.5341
Epoch 8/500
21/21 [==============================] - 0s 8ms/step - loss: 1.3259 - spar

In [43]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(1437, 5) (1437,)
(709, 5) (709,)
(1437, 5, 1)
Epoch 1/500
12/12 [==============================] - 0s 11ms/step - loss: 2.3462 - sparse_categorical_accuracy: 0.2422
Epoch 2/500
12/12 [==============================] - 0s 7ms/step - loss: 2.0927 - sparse_categorical_accuracy: 0.3132
Epoch 3/500
12/12 [==============================] - 0s 8ms/step - loss: 1.9671 - sparse_categorical_accuracy: 0.3681
Epoch 4/500
12/12 [==============================] - 0s 8ms/step - loss: 1.8267 - sparse_categorical_accuracy: 0.4029
Epoch 5/500
12/12 [==============================] - 0s 8ms/step - loss: 1.7376 - sparse_categorical_accuracy: 0.4322
Epoch 6/500
12/12 [==============================] - 0s 8ms/step - loss: 1.6465 - sparse_categorical_accuracy: 0.4468
Epoch 7/500
12/12 [==============================] - 0s 7ms/step - loss: 1.5974 - sparse_categorical_accuracy: 0.4495
Epoch 8/500
12/12 [==============================] - 0s 8ms/step - loss: 1.5800 - sparse

In [44]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
(13966, 5, 1)
Epoch 1/500
110/110 [==============================] - 1s 9ms/step - loss: 1.0572 - sparse_categorical_accuracy: 0.6569
Epoch 2/500
110/110 [==============================] - 1s 9ms/step - loss: 0.6100 - sparse_categorical_accuracy: 0.7859
Epoch 3/500
110/110 [==============================] - 1s 9ms/step - loss: 0.5342 - sparse_categorical_accuracy: 0.8060
Epoch 4/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4979 - sparse_categorical_accuracy: 0.8207
Epoch 5/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4743 - sparse_categorical_accuracy: 0.8267
Epoch 6/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4526 - sparse_categorical_accuracy: 0.8347
Epoch 7/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4380 - sparse_categorical_accuracy: 0.8361
Epoch 8/500
110/110 [==============================] - 1s 8ms/step - lo

In [45]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
(13966, 5, 1)
Epoch 1/500
110/110 [==============================] - 1s 8ms/step - loss: 0.9605 - sparse_categorical_accuracy: 0.6938
Epoch 2/500
110/110 [==============================] - 1s 8ms/step - loss: 0.6151 - sparse_categorical_accuracy: 0.7815
Epoch 3/500
110/110 [==============================] - 1s 8ms/step - loss: 0.5377 - sparse_categorical_accuracy: 0.8088
Epoch 4/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4897 - sparse_categorical_accuracy: 0.8231
Epoch 5/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4618 - sparse_categorical_accuracy: 0.8311
Epoch 6/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4470 - sparse_categorical_accuracy: 0.8359
Epoch 7/500
110/110 [==============================] - 1s 8ms/step - loss: 0.4200 - sparse_categorical_accuracy: 0.8468
Epoch 8/500
110/110 [==============================] - 1s 8ms/step - lo

In [46]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
(13966, 5, 1)
Epoch 1/500
110/110 [==============================] - 1s 9ms/step - loss: 0.9519 - sparse_categorical_accuracy: 0.6928
Epoch 2/500
110/110 [==============================] - 1s 9ms/step - loss: 0.6178 - sparse_categorical_accuracy: 0.7782
Epoch 3/500
110/110 [==============================] - 1s 9ms/step - loss: 0.5507 - sparse_categorical_accuracy: 0.7974
Epoch 4/500
110/110 [==============================] - 1s 9ms/step - loss: 0.5101 - sparse_categorical_accuracy: 0.8095
Epoch 5/500
110/110 [==============================] - 1s 9ms/step - loss: 0.4826 - sparse_categorical_accuracy: 0.8202
Epoch 6/500
110/110 [==============================] - 1s 9ms/step - loss: 0.4634 - sparse_categorical_accuracy: 0.8297
Epoch 7/500
110/110 [==============================] - 1s 9ms/step - loss: 0.4481 - sparse_categorical_accuracy: 0.8358
Epoch 8/500
110/110 [==============================] - 1s 9ms/step - lo

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(7045, 5) (7045,)
(3471, 5) (3471,)
(7045, 5, 1)
Epoch 1/500
56/56 [==============================] - 0s 9ms/step - loss: 1.1561 - sparse_categorical_accuracy: 0.6449
Epoch 2/500
56/56 [==============================] - 0s 8ms/step - loss: 0.7439 - sparse_categorical_accuracy: 0.7515
Epoch 3/500
56/56 [==============================] - 0s 8ms/step - loss: 0.6365 - sparse_categorical_accuracy: 0.7747
Epoch 4/500
56/56 [==============================] - 0s 9ms/step - loss: 0.5865 - sparse_categorical_accuracy: 0.7955
Epoch 5/500
56/56 [==============================] - 0s 8ms/step - loss: 0.5734 - sparse_categorical_accuracy: 0.7983
Epoch 6/500
56/56 [==============================] - 0s 8ms/step - loss: 0.5348 - sparse_categorical_accuracy: 0.8084
Epoch 7/500
56/56 [==============================] - 0s 8ms/step - loss: 0.5079 - sparse_categorical_accuracy: 0.8133
Epoch 8/500
56/56 [==============================] - 0s 8ms/step - loss: 0.4865 - sparse

In [11]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
(761, 8, 1)
Epoch 1/500
6/6 [==============================] - 0s 13ms/step - loss: 1.7382 - sparse_categorical_accuracy: 0.2602
Epoch 2/500
6/6 [==============================] - 0s 10ms/step - loss: 1.5323 - sparse_categorical_accuracy: 0.4192
Epoch 3/500
6/6 [==============================] - 0s 11ms/step - loss: 1.4236 - sparse_categorical_accuracy: 0.4744
Epoch 4/500
6/6 [==============================] - 0s 10ms/step - loss: 1.3204 - sparse_categorical_accuracy: 0.5375
Epoch 5/500
6/6 [==============================] - 0s 11ms/step - loss: 1.2401 - sparse_categorical_accuracy: 0.5427
Epoch 6/500
6/6 [==============================] - 0s 10ms/step - loss: 1.1613 - sparse_categorical_accuracy: 0.5940
Epoch 7/500
6/6 [==============================] - 0s 10ms/step - loss: 1.1135 - sparse_categorical_accuracy: 0.5664
Epoch 8/500
6/6 [==============================] - 0s 10ms/step - loss: 1.0794 - sparse_categ

In [12]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
(761, 8, 1)
Epoch 1/500
6/6 [==============================] - 0s 12ms/step - loss: 1.7028 - sparse_categorical_accuracy: 0.2878
Epoch 2/500
6/6 [==============================] - 0s 15ms/step - loss: 1.5293 - sparse_categorical_accuracy: 0.4113
Epoch 3/500
6/6 [==============================] - 0s 9ms/step - loss: 1.4162 - sparse_categorical_accuracy: 0.4757
Epoch 4/500
6/6 [==============================] - 0s 9ms/step - loss: 1.3329 - sparse_categorical_accuracy: 0.4612
Epoch 5/500
6/6 [==============================] - 0s 9ms/step - loss: 1.2460 - sparse_categorical_accuracy: 0.5191
Epoch 6/500
6/6 [==============================] - 0s 9ms/step - loss: 1.1983 - sparse_categorical_accuracy: 0.5361
Epoch 7/500
6/6 [==============================] - 0s 9ms/step - loss: 1.1319 - sparse_categorical_accuracy: 0.5624
Epoch 8/500
6/6 [==============================] - 0s 10ms/step - loss: 1.1245 - sparse_categorica

In [13]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(412, 7) (412,)
(204, 7) (204,)
(412, 7, 1)
Epoch 1/500
4/4 [==============================] - 0s 9ms/step - loss: 1.8453 - sparse_categorical_accuracy: 0.1820
Epoch 2/500
4/4 [==============================] - 0s 8ms/step - loss: 1.6404 - sparse_categorical_accuracy: 0.3981
Epoch 3/500
4/4 [==============================] - 0s 8ms/step - loss: 1.5468 - sparse_categorical_accuracy: 0.4490
Epoch 4/500
4/4 [==============================] - 0s 7ms/step - loss: 1.4622 - sparse_categorical_accuracy: 0.5024
Epoch 5/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3692 - sparse_categorical_accuracy: 0.5000
Epoch 6/500
4/4 [==============================] - 0s 8ms/step - loss: 1.3251 - sparse_categorical_accuracy: 0.5340
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 1.2807 - sparse_categorical_accuracy: 0.5364
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 1.2453 - sparse_categorical_a

In [14]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
(761, 8, 1)
Epoch 1/500
6/6 [==============================] - 0s 10ms/step - loss: 1.7662 - sparse_categorical_accuracy: 0.2431
Epoch 2/500
6/6 [==============================] - 0s 10ms/step - loss: 1.5977 - sparse_categorical_accuracy: 0.3811
Epoch 3/500
6/6 [==============================] - 0s 10ms/step - loss: 1.4686 - sparse_categorical_accuracy: 0.4376
Epoch 4/500
6/6 [==============================] - 0s 10ms/step - loss: 1.3759 - sparse_categorical_accuracy: 0.4836
Epoch 5/500
6/6 [==============================] - 0s 9ms/step - loss: 1.2832 - sparse_categorical_accuracy: 0.5519
Epoch 6/500
6/6 [==============================] - 0s 11ms/step - loss: 1.2168 - sparse_categorical_accuracy: 0.5493
Epoch 7/500
6/6 [==============================] - 0s 9ms/step - loss: 1.1592 - sparse_categorical_accuracy: 0.5637
Epoch 8/500
6/6 [==============================] - 0s 10ms/step - loss: 1.0924 - sparse_categor

In [15]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
(2526, 10, 1)
Epoch 1/500
20/20 [==============================] - 0s 14ms/step - loss: 3.0280 - sparse_categorical_accuracy: 0.0772
Epoch 2/500
20/20 [==============================] - 0s 13ms/step - loss: 2.9342 - sparse_categorical_accuracy: 0.1053
Epoch 3/500
20/20 [==============================] - 0s 13ms/step - loss: 2.8412 - sparse_categorical_accuracy: 0.1211
Epoch 4/500
20/20 [==============================] - 0s 12ms/step - loss: 2.7468 - sparse_categorical_accuracy: 0.1492
Epoch 5/500
20/20 [==============================] - 0s 13ms/step - loss: 2.6561 - sparse_categorical_accuracy: 0.1805
Epoch 6/500
20/20 [==============================] - 0s 13ms/step - loss: 2.5762 - sparse_categorical_accuracy: 0.1920
Epoch 7/500
20/20 [==============================] - 0s 13ms/step - loss: 2.4822 - sparse_categorical_accuracy: 0.2233
Epoch 8/500
20/20 [==============================] - 0s 14ms/step - lo

In [16]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
(2526, 10, 1)
Epoch 1/500
20/20 [==============================] - 0s 13ms/step - loss: 3.0188 - sparse_categorical_accuracy: 0.0681
Epoch 2/500
20/20 [==============================] - 0s 12ms/step - loss: 2.9355 - sparse_categorical_accuracy: 0.1041
Epoch 3/500
20/20 [==============================] - 0s 11ms/step - loss: 2.8450 - sparse_categorical_accuracy: 0.1409
Epoch 4/500
20/20 [==============================] - 0s 13ms/step - loss: 2.7678 - sparse_categorical_accuracy: 0.1429
Epoch 5/500
20/20 [==============================] - 0s 12ms/step - loss: 2.6603 - sparse_categorical_accuracy: 0.1801
Epoch 6/500
20/20 [==============================] - 0s 12ms/step - loss: 2.5686 - sparse_categorical_accuracy: 0.2126
Epoch 7/500
20/20 [==============================] - 0s 12ms/step - loss: 2.4991 - sparse_categorical_accuracy: 0.2189
Epoch 8/500
20/20 [==============================] - 0s 12ms/step - lo

In [17]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
(2526, 10, 1)
Epoch 1/500
20/20 [==============================] - 0s 12ms/step - loss: 3.0554 - sparse_categorical_accuracy: 0.0637
Epoch 2/500
20/20 [==============================] - 0s 12ms/step - loss: 2.9791 - sparse_categorical_accuracy: 0.0974
Epoch 3/500
20/20 [==============================] - 0s 12ms/step - loss: 2.9106 - sparse_categorical_accuracy: 0.1168
Epoch 4/500
20/20 [==============================] - 0s 12ms/step - loss: 2.8239 - sparse_categorical_accuracy: 0.1346
Epoch 5/500
20/20 [==============================] - 0s 12ms/step - loss: 2.7290 - sparse_categorical_accuracy: 0.1587
Epoch 6/500
20/20 [==============================] - 0s 12ms/step - loss: 2.6371 - sparse_categorical_accuracy: 0.1821
Epoch 7/500
20/20 [==============================] - 0s 13ms/step - loss: 2.5503 - sparse_categorical_accuracy: 0.2031
Epoch 8/500
20/20 [==============================] - 0s 12ms/step - lo

In [18]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(1372, 10) (1372,)
(676, 10) (676,)
(1372, 10, 1)
Epoch 1/500
11/11 [==============================] - 0s 13ms/step - loss: 3.0470 - sparse_categorical_accuracy: 0.0554
Epoch 2/500
11/11 [==============================] - 0s 11ms/step - loss: 2.9740 - sparse_categorical_accuracy: 0.0904
Epoch 3/500
11/11 [==============================] - 0s 11ms/step - loss: 2.9251 - sparse_categorical_accuracy: 0.1093
Epoch 4/500
11/11 [==============================] - 0s 11ms/step - loss: 2.8785 - sparse_categorical_accuracy: 0.1195
Epoch 5/500
11/11 [==============================] - 0s 11ms/step - loss: 2.8219 - sparse_categorical_accuracy: 0.1509
Epoch 6/500
11/11 [==============================] - 0s 11ms/step - loss: 2.7986 - sparse_categorical_accuracy: 0.1458
Epoch 7/500
11/11 [==============================] - 0s 11ms/step - loss: 2.7285 - sparse_categorical_accuracy: 0.1611
Epoch 8/500
11/11 [==============================] - 0s 11ms/step - loss